In [165]:
#You can find my project side here: https://leeeeeeapgn.github.io/Project1_Lede/
#I am going to look at a dataset from University of Rochester about Chinese leaders visiting other countries and look for patterns.
#I ended up doing a few analysis but not using all of them for the story. 

In [213]:
import pandas as pd
print(pd.__version__)


2.3.0


In [214]:
import pandas as pd

df = pd.read_stata("china_visits.v1.dta", convert_categoricals=False)

print(df.head())

              country  year ccode             start               end  \
0          Kazakhstan  1998   KAZ       July 3 1998       July 4 1998   
1  Russian Federation  1998   RUS  November 22 1998  November 25 1998   
2               Japan  1998   JPN  November 25 1998  November 30 1998   
3               Italy  1999   ITA     March 20 1999     March 25 1999   
4         Switzerland  1999   CHE     March 25 1999     March 27 1999   

   group_visit  group_order  group_id       leader  leader_id     office  \
0          0.0          1.0       1.0  Jiang Zemin        1.0  President   
1          1.0          1.0       2.0  Jiang Zemin        1.0  President   
2          1.0          2.0       2.0  Jiang Zemin        1.0  President   
3          1.0          1.0       3.0  Jiang Zemin        1.0  President   
4          1.0          2.0       3.0  Jiang Zemin        1.0  President   

                                                 url   last_accessed  \
0  http://www.peopledaily.com.cn

In [215]:
#I am importing the csv file. I was unable to convert the original data and needed help from another user. 
#The person did the exact same thing, so unclear what went wrong. I imported his csv.

In [216]:
import pandas as pd

df = pd.read_csv("china_visits.v1.csv")

print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'china_visits.v1.csv'

In [217]:
print(df.columns)

Index(['country', 'year', 'ccode', 'start', 'end', 'group_visit',
       'group_order', 'group_id', 'leader', 'leader_id', 'office', 'url',
       'last_accessed', 'meeting', 'note'],
      dtype='object')


In [218]:
df.shape

(405, 15)

In [219]:
df.dtypes


country           object
year               int16
ccode             object
start             object
end               object
group_visit      float32
group_order      float32
group_id         float32
leader            object
leader_id        float32
office            object
url               object
last_accessed     object
meeting           object
note              object
dtype: object

In [220]:
df.country

0              Kazakhstan
1      Russian Federation
2                   Japan
3                   Italy
4             Switzerland
              ...        
400             Singapore
401               Croatia
402    Russian Federation
403            Uzbekistan
404              Thailand
Name: country, Length: 405, dtype: object

In [221]:
trip_countries = df['country'].value_counts()
print(trip_countries)

country
Russian Federation    22
Kazakhstan            14
Germany               13
United Kingdom         9
France                 9
                      ..
Zambia                 1
Namibia                1
Mozambique             1
Seychelles             1
Colombia               1
Name: count, Length: 111, dtype: int64


In [222]:
#I wanted to take a look at the whole list for once to get a feeling on what the data overall looked like and what would make sense to show.

In [223]:
trip_countries = df['country'].value_counts()
print(trip_countries)

country
Russian Federation    22
Kazakhstan            14
Germany               13
United Kingdom         9
France                 9
                      ..
Zambia                 1
Namibia                1
Mozambique             1
Seychelles             1
Colombia               1
Name: count, Length: 111, dtype: int64


In [224]:
trip_countries = df['country'].value_counts().reset_index()
trip_countries.columns = ['Country', 'Number of Visits']
trip_countries.to_csv("trip_countries.csv", index=False)

In [225]:
top10 = trip_countries.head(10)
top10.to_csv("top10_countries.csv", index=False)

In [226]:
sco_members = [
    "China", "India", "Kazakhstan", "Kyrgyzstan", "Pakistan",
    "Russian Federation", "Tajikistan", "Uzbekistan", "Iran"
]

In [227]:
sco_visits = df[df['country'].isin(sco_members)]
sco_counts = sco_visits['country'].value_counts().reset_index()
sco_counts.columns = ['Country', 'Number of Visits']

In [228]:
sco_counts.to_csv("sco_counts.csv", index=False)

In [229]:
asean_members = [
    "Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia",
    "Myanmar", "Philippines", "Singapore", "Thailand", "Vietnam"
]

In [230]:
asean_by_year = df[df['country'].isin(asean_members)].groupby(['country', 'year']).size().reset_index(name='Number of Visits')
asean_by_year.columns = ['Country', 'Year', 'Number of Visits']
asean_by_year.to_csv("asean_visits_by_year.csv", index=False)

In [231]:
asean_yearly = asean_by_year.groupby('Year')['Number of Visits'].sum().reset_index()
asean_yearly.to_csv("asean_total_per_year.csv", index=False)


In [232]:
sco_by_year = df[df['country'].isin(sco_members)].groupby(['country', 'year']).size().reset_index(name='Number of Visits')
sco_by_year.columns = ['Country', 'Year', 'Number of Visits']
sco_by_year.to_csv("sco_by_year.csv", index=False)



In [239]:
import pandas as pd

url = "https://en.wikipedia.org/wiki/Shanghai_Cooperation_Organisation"
tables = pd.read_html(url, match="Summits of heads of state")
chair_df = tables[0][["Date", "Country"]].copy()
chair_df["Year"] = chair_df["Date"].str.extract(r'(\d{4})').astype(int)
chair_df = chair_df[["Year", "Country"]]
print(chair_df)

#chatgpt helped me here

    Year     Country
0   2001       China
1   2002      Russia
2   2003      Russia
3   2004  Uzbekistan
4   2005  Kazakhstan
5   2006       China
6   2007  Kyrgyzstan
7   2008  Tajikistan
8   2009      Russia
9   2010  Uzbekistan
10  2011  Kazakhstan
11  2012       China
12  2013  Kyrgyzstan
13  2014  Tajikistan
14  2015      Russia
15  2016  Uzbekistan
16  2017  Kazakhstan
17  2018       China
18  2019  Kyrgyzstan
19  2020      Russia
20  2021  Tajikistan
21  2022  Uzbekistan
22  2023       India
23  2024  Kazakhstan
24  2025       China


In [241]:
merged = chair_df.merge(visits, left_on=["Year", "Country"], right_on=["Year", "Country"], how="left")
merged["Visit_to_Chair"] = merged["Number of Visits"].notna()
merged[["Year", "Country", "Number of Visits", "Visit_to_Chair"]].to_csv("sco_chair_comparison.csv", index=False)
merged.to_csv("sco_merged.csv", index=False)

In [ ]:
#I ended up not using the rest, because it was leaving away from my story. 

In [ ]:
germany_trips = df[df['country'] == "Germany"]
print(germany_trips)

In [186]:
germany_trips[["year", "leader", "office"]]

,year,leader,office
39,2002,Jiang Zemin,President
77,2005,Hu Jintao,President
151,2014,Xi Jinping,President
192,2017,Xi Jinping,President
230,2000,Zhu Rongji,Premier
264,2004,Wen Jiabao,Premier
296,2006,Wen Jiabao,Premier
310,2009,Wen Jiabao,Premier
331,2011,Wen Jiabao,Premier
353,2013,Li Keqiang,Premier


In [187]:
germany_trips_sorted = germany_trips.sort_values(by="year")
print(germany_trips_sorted[["year", "leader", "office"]])


     year       leader     office
230  2000   Zhu Rongji    Premier
39   2002  Jiang Zemin  President
264  2004   Wen Jiabao    Premier
77   2005    Hu Jintao  President
296  2006   Wen Jiabao    Premier
310  2009   Wen Jiabao    Premier
331  2011   Wen Jiabao    Premier
353  2013   Li Keqiang    Premier
151  2014   Xi Jinping  President
364  2014   Li Keqiang    Premier
192  2017   Xi Jinping  President
388  2017   Li Keqiang    Premier
396  2018   Li Keqiang    Premier


In [188]:
print(trip_countries["United States"])

KeyError: 'United States'